In [3]:
import glob
import matplotlib.pyplot as plt
import os
import h5py 
import numpy as np

In [27]:
os.getcwd()

'C:\\Users\\eac84\\Universiteit Antwerpen\\Thermography - Biomedical thermography\\src'

In [17]:
import os
os.chdir("..")

In [3]:
%load_ext line_profiler
%load_ext autoreload
%autoreload 2
%matplotlib qt

In [20]:
from thermography_python import read_data

In [4]:
import glob

In [1]:
root = "T:\ProjectData\UZADIEPFlap\2021-09-01 - Pre-op_link_with_CT/*/"

SyntaxError: (unicode error) 'unicodeescape' codec can't decode bytes in position 14-15: truncated \UXXXXXXXX escape (<ipython-input-1-f2e1c1ab1cff>, line 1)

In [21]:
def export_xvis(root):
    xvi_fns = glob.glob(root + "/*.xvi")
    for fn in xvi_fns:
        export_xvi(fn)

def export_datasets(root):

    H5_fns = glob.glob(root + "\\*.H5")
    print(H5_fns)
    for fn in H5_fns:  
        export_xvi(fn)
    
def export_xvifile(fn,fn_out=None):
    
    if fn_out is None:
        fn_out = fn.replace(".xvi",".H5").replace(".XVI",".H5")
    
    XVI = read_data.XviReader(fn)
    samp_rate = 10
    samples = np.arange(0,len(XVI),samp_rate)
    
    if len(XVI)==0: return 
    im_data = np.array([XVI[fr] for fr in samples])
        
    minx = np.min((im_data[0],im_data[-1]))
    maxx = np.max((im_data[0],im_data[-1]))
    limits = [minx,maxx]
    
    im_data = reduce_uint16(im_data,limits)
    
    attributes = {'FPS': 1, 
                  'camera_model':'XENIX', 
                  "limits":limits,
                  "sample_rate":samp_rate}

    save_h5(fn_out , im_data, attributes=attributes)
    
def export_H5file(fn):
        
    print(fn)
    file_parts = fn.split("\\")
    date = file_parts[-2].replace("_","")
    name = file_parts[-1].replace(" ","-").replace("_","").replace(".H5","").replace(".","-")
    fn_out = root+ "\\" + date + "_" + name + ".H5"
    
    with h5py.File(fn, 'r') as fh:  
        
        if "im_data" not in fh: return
            
        attributes = {}
        for k in fh["im_data"].attrs.keys(): attributes[k] = fh["im_data"].attrs[k]            
            
    #if attributes["FPS"] == 1: continue 
                
    with h5py.File(fn, 'r') as fh:  
        
        im = fh["im_data"][::samp]
                
    im = reduce_uint16(im)
    
    attributes = {'FPS': 1, 'camera_model': 'XENIX', 
                  limits:[9000,17000], "sample_rate":samp}
        
    save_h5(fn_out , im, attributes=attributes)
    
    
def reduce_uint16(im, limits):
    
    #limits = 9000,17000# it was 16000
    lo,hi = limits
        
    if im.dtype == np.uint16:    
        im = 1*.im - lo
        im = im * 255./(hi-lo)
        im = im.clip(0,255)
        im = im.astype(np.uint8)

    return im

def save_h5(fn, im, attributes=None):  
    
    with h5py.File(fn, 'a') as fh:
        if "im_data" in fh.keys(): del fh["im_data"]
        dset = fh.create_dataset("im_data", data=im , compression="lzf" )
        if attributes is not None:
            for key in attributes:
                dset.attrs[key] = attributes[key]
    
    

In [85]:
root = "C:/Users/eac84/Desktop/Test_Flap_data/"
H5_fns = glob.glob(root +  "2*.H5")
H5_fns = [fn.split("\\")[-1][:8] for fn in H5_fns]
H5_fns
##########
root2 = "T:\ProjectData\\"
xvi_fns = glob.glob(root2+"UZADIEPFlap\\20**\\*pre*.xvi",recursive=True)
##########

In [25]:
##########

#xvi_fns = glob.glob( "T:/ProjectData/UZADIEPFlap/20*/**/*pre*.xvi",recursive=True)
xvi_fns = glob.glob( "T:/ProjectData/UZADIEPFlap/2021-09-01 - Pre-op_link_with_CT/**/*pre*.xvi",recursive=True)
out_dir = "I:/3_phd/Edgar/DIEP FLAP/PRE"

In [26]:
xvi_fns

['T:/ProjectData/UZADIEPFlap/2021-09-01 - Pre-op_link_with_CT\\patient1\\precooling_link_with_CT_Patient1.xvi',
 'T:/ProjectData/UZADIEPFlap/2021-09-01 - Pre-op_link_with_CT\\patient2\\Pre_OP_link_with_CT_Patient2.xvi',
 'T:/ProjectData/UZADIEPFlap/2021-09-01 - Pre-op_link_with_CT\\patient3\\Pre_OP_link_with_CT_patient3.xvi',
 'T:/ProjectData/UZADIEPFlap/2021-09-01 - Pre-op_link_with_CT\\patient4\\Pre_OP_link_with_CT_Patient4.xvi']

In [27]:
for n,fn in enumerate(xvi_fns):
    
    x = fn.split("\\")[-2].split(" - ")[0].replace("-","").replace("_","-")
    x = x + "_"+ fn.split("\\")[-1].replace(" ","-").replace(".xvi","").replace(".","")
    x = x.replace("pre1","pre") + ".H5"
    x = out_dir+ "/"  + x.upper()
    
    #if x[:8] in H5_fns: continue 
    print(n)
    print(x)
        
    export_xvifile(fn,x)

0
I:/3_phd/Edgar/DIEP FLAP/PRE/PATIENT1_PRECOOLING_LINK_WITH_CT_PATIENT1.H5
1
I:/3_phd/Edgar/DIEP FLAP/PRE/PATIENT2_PRE_OP_LINK_WITH_CT_PATIENT2.H5
2
I:/3_phd/Edgar/DIEP FLAP/PRE/PATIENT3_PRE_OP_LINK_WITH_CT_PATIENT3.H5
3
I:/3_phd/Edgar/DIEP FLAP/PRE/PATIENT4_PRE_OP_LINK_WITH_CT_PATIENT4.H5


In [52]:
10*60*10

6000

###### trmo_data = ThermoData(fn)
trmo_data.save_H5()

In [226]:
import read_data as read

In [254]:
plt.imshow(im_data[-1])

In [278]:
im_data = np.array(im_data)

In [279]:
plt.plot(im_data[:,200,200])

In [149]:
import numpy as np

In [32]:
%matplotlib qt

In [ ]:
import h5py

In [ ]:
root = "T:\ProjectData\\DIEP FLAP\\"

In [205]:
root = "C:\\Users\\eac84\\Desktop\\"

['C:\\Users\\eac84\\Desktop\\pre op met cooling.H5', 'C:\\Users\\eac84\\Desktop\\strm_data.h5']
C:\Users\eac84\Desktop\pre op met cooling.H5
{'FPS': 30, 'camera_model': 'XENIX'}
C:\Users\eac84\Desktop\strm_data.h5


In [268]:

im2 = reduce_uint16(np.array(im_data))

In [266]:
np.array(im_data)

In [1]:
plt.figure()
plt.imshow(im_data[1].clip(0,17000))
plt.figure()
plt.imshow(im2[1])

NameError: name 'plt' is not defined

In [187]:
root = "T:\ProjectData\\DIEP FLAP\\"

In [188]:
H5_fns = glob.glob(root+ "*.H5")

In [191]:
fn = H5_fns[10]
with h5py.File(fn, 'r') as fh:  
    im = fh["im_data"][:]